In [1]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
    logging,
)



In [2]:
# model_name = "/data/opt-350m"
# model_name = "/data/llama-hf/Llama-2-7b-hf"
# model_name = "/data/Mistral-7B-Instruct-v0.2"
# model_name = "mistralai/Mistral-7B-v0.1"
model_name = "codellama/CodeLlama-13b-Python-hf"
model_name = "bigcode/starcoder"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
device_map = {"": 0}
# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.eval()

model.config.use_cache = False
model.config.pretraining_tp = 1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if "Llama-2" in model_name or "Mistral" in model_name:
    tokenizer.pad_token = tokenizer.bos_token

bos = tokenizer.bos_token_id
eos = tokenizer.eos_token_id
pad = tokenizer.pad_token_id
print("pre-trained model's BOS EOS and PAD token id:",bos,eos,pad," => It should be 1 2 None")

##### tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
tokenizer.padding_side = "right"


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/36.3k [00:00<?, ?B/s]

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

pytorch_model-00002-of-00007.bin:   0%|          | 0.00/9.86G [00:00<?, ?B/s]

pytorch_model-00003-of-00007.bin:   0%|          | 0.00/9.85G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/532 [00:00<?, ?B/s]

pre-trained model's BOS EOS and PAD token id: 0 0 None  => It should be 1 2 None


In [18]:
# prompt = "def fibonoacci("
# prompt = '''def quicksort(a: List(int)) -> List:
#     """ <FILL_ME>
#     return result
# '''

prompt = '''def SwapGate(q1,q2) -> List:
    """ <FILL_ME>
    return result
'''
# print(prompt)
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length= 300)
# result = pipe(f"{prompt}")
# print(result[0]['generated_text'])

input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to("cuda")
output = model.generate(
    input_ids,
    max_new_tokens=200,
)
output = output[0].to("cpu")

filling = tokenizer.decode(output[input_ids.shape[1]:], skip_special_tokens=True)
print(prompt.replace("<FILL_ME>", filling))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


def quicksort(a: List(int)) -> List:
    """ Quicksort algorithm.

    Args:
        a (List(int)): List to be sorted.

    Returns:
        List: Sorted list.
    """
    if len(a) <= 1:
        return a
    pivot = a[0]
    less = [x for x in a[1:] if x < pivot]
    greater = [x for x in a[1:] if x >= pivot]
    result = quicksort(less) + [pivot] + quicksort(greater)
    return result



In [33]:
# prompt = 'def remove_non_ascii(s: str) -> str:\n    """ '
system = "Provide answers in Python"
# user = "Write a quantum circuit to swap two qubits using qiskit."
user = "Design a quantum circuit to teleport q2 to q1."

prompt = f"<s>[INST] <<SYS>>\\n{system}\\n<</SYS>>\\n\\n{user}[/INST]"
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to("cuda")

# inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

output = model.generate(
    inputs["input_ids"],
    max_new_tokens=300,
    do_sample=True,
    top_p=0.9,
    temperature=0.1,
)
output = output[0].to("cpu")
print(tokenizer.decode(output))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] <<SYS>>\nProvide answers in Python\n<</SYS>>\n\nDesign a quantum circuit to teleport q2 to q1.[/INST]

[TITLE]Quantum Teleportation[/TITLE]

[SYNTAX]

[CODE]

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit import Aer

q = QuantumRegister(3)
c = ClassicalRegister(3)
qc = QuantumCircuit(q, c)

qc.h(q[1])
qc.cx(q[1], q[2])
qc.cx(q[1], q[0])
qc.h(q[0])
qc.measure(q[0], c[0])
qc.measure(q[1], c[1])
qc.z(q[2]).c_if(c, 2)
qc.measure(q[2], c[2])

backend_sim = Aer.get_backend('qasm_simulator')
job_sim = execute(qc, backend_sim)
result_sim = job_sim.result()

print(result_sim.get_counts(qc))

[/CODE]

[/SYNTAX]

[EXPLAIN]

[/EXPLAIN]

[


In [8]:
# # prompt = 'def remove_non_ascii(s: str) -> str:\n    """ '
# system = "Provide answers in Python"
# # user = "Write a quantum circuit to swap two qubits using qiskit."
# user = "Design a quantum circuit to teleport q2 to q1."

# prompt = f"<s>[INST] <<SYS>>\\n{system}\\n<</SYS>>\\n\\n{user}[/INST]"
prompt = "# function to generate a quantum circuit to do quantum teleportation from q0 to q2:\n"
prompt = "import cudaq \n\n def teleport_measure(q0,q1,q2)"
inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to("cuda")

# inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

output = model.generate(
    inputs["input_ids"],
    max_new_tokens=300,
    do_sample=True,
    top_p=0.9,
    temperature=0.1,
)
output = output[0].to("cpu")
print(tokenizer.decode(output))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


import cudaq 

 def teleport_measure(q0,q1,q2) :
  q0.H()
  q1.H()
  q2.H()
  q0.cnot(q1)
  q1.cnot(q2)
  q2.cnot(q0)
  q0.H()
  q1.H()
  q2.H()
  return q0.measure()

 def teleport_measure_all(q0,q1,q2) :
  q0.H()
  q1.H()
  q2.H()
  q0.cnot(q1)
  q1.cnot(q2)
  q2.cnot(q0)
  q0.H()
  q1.H()
  q2.H()
  return q0.measure(),q1.measure(),q2.measure()

 def teleport_measure_all_q(q0,q1,q2) :
  q0.H()
  q1.H()
  q2.H()
  q0.cnot(q1)
  q1.cnot(q2)
  q2.cnot(q0)
  q0.H()
  q1.H()
  q2.H()
  return q0,q1,q2

 def teleport_measure_all_q_all(q0,q1,q2) :
  q0.


In [3]:
# prompt = "Write a quantum circuit to swap two qubits."
# prompt = "def SwapGate(q1, q2):"
prompt = "import cudaq"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens= 200)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

/opt/conda/envs/mytorch/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


import cudaq
from cudaq.cuda import Cuda
from cudaq.cudart import CudaRuntime
from cudaq.cublas import Cublas
from cudaq.cufft import Cufft
from cudaq.cufft import CufftPlan1d
from cudaq.cufft import CufftPlan2d
from cudaq.cufft import CufftPlan3d
from cudaq.cufft import CufftPlanNd
from cudaq.cufft import CufftPlanMany
from cudaq.cufft import CufftExec
from cudaq.cufft import CufftExecC2C
from cudaq.cufft import CufftExecR2C
from cudaq.cufft import CufftExecC2R
from cudaq.c
